### Check $V_{rms}$ function

In [98]:
# # Constant viscosity convection, Cartesian domain (benchmark)
#
#
#
# This example solves 2D dimensionless isoviscous thermal convection with a Rayleigh number, for comparison with the [Blankenbach et al. (1989) benchmark](https://academic.oup.com/gji/article/98/1/23/622167).
#
# We set up a v, p, T system in which we will solve for a steady-state T field in response to thermal boundary conditions and then use the steady-state T field to compute a stokes flow in response.

# +tolerace
import os
os.environ["UW_TIMING_ENABLE"] = "1"
import petsc4py
from petsc4py import PETSc


import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

import numpy as np
import sympy


In [99]:
res = 48

In [100]:
# +
meshbox = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0), maxCoords=(1.0, 1.0), cellSize=1.0 / res, regular=False, qdegree=3
)

v_soln = uw.discretisation.MeshVariable("U", meshbox, meshbox.dim, degree=2)

Processing gmsh file .meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(1.0, 1.0)_csize0.020833333333333332_regFalse.msh
Mesh saved to .meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(1.0, 1.0)_csize0.020833333333333332_regFalse.msh.h5


In [101]:
# underworld3 function for calculating the rms velocity 
import math

def v_rms(mesh = meshbox, v_solution = v_soln): 
    # v_soln must be a variable of mesh
    v_rms = math.sqrt(uw.maths.Integral(mesh, v_solution.fn.dot(v_solution.fn)).evaluate())
    #v_rms = uw.maths.Integral(mesh, v_solution.fn.dot(v_solution.fn)).evaluate()
    return v_rms


In [106]:
x = sympy.Symbol('x')
z = sympy.Symbol('z')
# fx  = sympy.exp(x)
# fz = sympy.exp(z)
fx  = sympy.cos(x)
fz = sympy.sin(z)
int1 = sympy.integrate(fx*fx + fz*fz, (x, 0, 1))
final_int = sympy.integrate(int1, (z, 0, 1))
final_int

1

In [109]:
with meshbox.access(v_soln):
    for i, coord in enumerate(v_soln.coords):
        v_soln.data[i, 0] = math.exp(coord[0])
        v_soln.data[i, 1] = math.exp(coord[1])
        v_soln.data[i, 0] = math.cos(coord[0])
        v_soln.data[i, 1] = math.sin(coord[1])
        #print(coord)

In [110]:
v_rms(meshbox, v_soln)

0.99999999989184

In [111]:
# the analytical solution
exp_vrms = math.sqrt(math.exp(2) - 1)
print(exp_vrms)

2.5276582243117147
